### 爬取新闻

In [ ]:
import requests
import json
res = requests.get('http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gjxw&level==1||=2&show_ext=1&show_all=1&show_num=500&tag=1&format=json&page=1&callback=newsloadercallback&_=1496538497721')
#res.text
# lstrip将字符串的左边去掉，rstrip将字符串的右边去掉，得到干净的数据
jd = res.text.lstrip('  newsloadercallback(').rstrip(');')
# 变成json数据，方便提取内容
data = json.loads(jd)

### 建立新闻内爬取函数

In [ ]:
from bs4 import BeautifulSoup
def getArticle(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    ## 分开的词用空格连接
    return ' '.join([p.text.strip() for p in soup.select('#artibody p')])

In [ ]:
ary = []
for rec in data['result']['data']:
    try:
        ary.append({'title':rec['ext3'], 'url':rec['url'], 'content': getArticle(rec['url'])})
    except:
        print(rec['ext3'])

In [ ]:
import pandas
# 变成dataframe格式就可以方便的存储为excel类型
df = pandas.DataFrame(ary)
df.to_excel('news.xlsx')

In [4]:
import pandas
df = pandas.read_excel('Data/news.xlsx',index_col=0) 
df.head()

,content,title,url
0,英国《每日电讯报》记者Harry Yorke 称，伦敦桥附近再次传出巨大爆炸声，爆炸强度是半...,伦敦桥附近再传爆炸声,http://news.sina.com.cn/w/zx/2017-06-04/doc-if...
1,[环球网综合报道]据英国《每日邮报》6月1日报道，近日，美国洛杉矶市比弗利山庄罗迪欧大道诊所...,美诊所1.5万人整形记录被盗,http://news.sina.com.cn/w/zx/2017-06-04/doc-if...
2,[环球网综合报道]据美国有线电视新闻网4日报道，当地时间3日，英国伦敦相继发生三起袭击事件，...,特朗普“发推”声援伦敦,http://news.sina.com.cn/o/2017-06-04/doc-ifyfu...
3,原标题：军情锐评 | 文在寅为何成“挺萨德派”？俄军对付美军有一套…… 近日，美国展示拦截洲...,文在寅为何成“挺萨德派”,http://news.sina.com.cn/w/zx/2017-06-04/doc-if...
4,中新网6月4日电 据外媒报道，在伦敦桥和巴罗市场3日遭恐袭后，有目击者称，在巴罗市场连续传来...,2名疑似袭击者被击毙,http://news.sina.com.cn/o/2017-06-04/doc-ifyfu...


In [1]:
! pip install jieba

Looking in indexes: https://pypi.doubanio.com/simple
                                              0.0/19.2 MB ? eta -:--:--
                                              0.0/19.2 MB ? eta -:--:--
                                              0.0/19.2 MB ? eta -:--:--
                                             0.0/19.2 MB 653.6 kB/s eta 0:00:30
                                             0.1/19.2 MB 648.1 kB/s eta 0:00:30
                                             0.1/19.2 MB 573.4 kB/s eta 0:00:34
                                             0.1/19.2 MB 654.9 kB/s eta 0:00:30
                                             0.2/19.2 MB 752.9 kB/s eta 0:00:26
                                             0.2/19.2 MB 778.2 kB/s eta 0:00:25
                                             0.2/19.2 MB 778.2 kB/s eta 0:00:25
                                             0.2/19.2 MB 778.2 kB/s eta 0:00:25
                                             0.2/19.2 MB 778.2 kB/s eta 0:00:25
           

In [8]:
import jieba
titles   = []
articles = [] 
for rec in df.iterrows():
   
     #print(rec[1].content)
    articles.append(' '.join(jieba.cut(rec[1].content)))
    titles.append(rec[1].title)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ZHESIW~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.735 seconds.
Prefix dict has been built successfully.


### 建立词频矩阵

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# 将文本转换成数值特征向量
# 每一行代表一篇文章，每一列代表一个词
# 在文章中存在的值词标为1，不存在标为0
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(articles)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# 文档之间的计算余弦相似度，就是距离
cosine_similarities  = cosine_similarity(X, X)

In [ ]:
from sklearn.cluster import KMeans
c = KMeans(n_clusters=10, init = 'k-means++', random_state=123)
k_data = c.fit_predict(cosine_similarities)

In [ ]:
import numpy
titles_ary = numpy.array(titles)
titles_ary[k_data == 9]